# 4. 文本转语音任务

- 文本转语音（Text-to-Speech，TTS）是指根据文本输入生成自然流畅语音的任务。TTS 模型可进一步扩展为单模型支持多说话人、多语言的语音生成系统。文本转语音也是语音合成。在Transformers中分成：
    - Text-to-Speech
    - Text-to-Audio

## 4.1. 文本转语音任务应用

- 在语音合成模型中，我们选择支持Transformers、魔塔与第三方等技术框架的模型,在transformers框架中实现的算法有：
    - Mimi
    - MMS
    - Moshi
    - Seamless-M4T
    - SeamlessM4T-v2
    - Speech2Text
    - Speech2Text2
    - SpeechT5
    - Wav2Vec2
    - Wav2Vec2-BERT
    - Wav2Vec2-Conformer
    - Wav2Vec2Phoneme
    - Whisper
- 在选择模型的时候可以根据这些算法关键字过滤。

### (1) 模型下载

- 微软（microsoft）模型
    - `git clone https://www.modelscope.cn/microsoft/speecht5_tts.git`

- facebook模型下载
    - `git clone https://www.modelscope.cn/facebook/hf-seamless-m4t-large.git`
    - `git clone https://www.modelscope.cn/facebook/musicgen-stereo-medium.git`(Text-to-Audio)

- Transformers站点推荐的：
    - `git clone https://www.modelscope.cn/ACE-Step/Ace-Step1.5.git`(Text-to-Audio)
    - `git clone https://www.modelscope.cn/mapjack/bark.git`

### (2) 使用speecht5_tts模型

In [1]:
from transformers import pipeline
from datasets import load_dataset
import soundfile as sf
import torch

pipe = pipeline("text-to-speech", model="F:/03Models/speecht5_tts")

embeddings_dataset = load_dataset("F:/04Datasets/cmu-arctic-xvectors", split="validation", trust_remote_code=True)
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
speech = pipe("Hello, my dog is cooler than you!", forward_params={"speaker_embeddings": speaker_embedding})
print(speech)
sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])

Device set to use cuda:0


{'audio': array([ 0.00052379,  0.00074588,  0.00091089, ..., -0.00042187,
       -0.00017906, -0.00015127], shape=(34816,), dtype=float32), 'sampling_rate': 16000}


### (3) 使用hf-seamless-m4t-large模型

In [2]:
from transformers import pipeline
import numpy as np
import soundfile as sf

pipe = pipeline("text-to-speech", model="F:/03Models/hf-seamless-m4t-large")
print(pipe)
speech = pipe("我是中国人！", forward_params={"tgt_lang": "cmn"})
print(speech)
# sf.write("speech_cmn.wav", np.squeeze(speech["audio"], axis=0), samplerate=speech["sampling_rate"])
print(speech["audio"][0].shape)
sf.write("speech_cmn.wav", speech["audio"][0], samplerate=speech["sampling_rate"])

Device set to use cuda:0


{'audio': array([[-1.7293938e-04, -1.8664182e-04, -1.4298852e-04, ...,
        -4.9232389e-05, -7.8199897e-05, -4.9895141e-05]],
      shape=(1, 24640), dtype=float32), 'sampling_rate': 16000}


### (4) 使用bark模型

- Bark是一个由Suno创建的基于变换器的文本转音频模型。
    - Bark能够生成高度逼真的多语言语音以及其他音频，包括音乐、背景噪音和简单的音效。
    - 该模型还能产生非言语交流，如笑声、叹息和哭泣。

In [1]:
from transformers import pipeline
import soundfile as sf
print("加载模型")
synthesiser = pipeline("text-to-speech", "F:/03Models/bark")
print("模型推理")
speech = synthesiser("Hello, my dog  is cooler than you! [laughs]", forward_params={"do_sample": True})
print("保存结果")
print(speech["audio"][0].shape)
sf.write("bark_out.wav", speech["audio"][0], samplerate=speech["sampling_rate"])

加载模型


The tokenizer you are loading from 'F:/03Models/bark' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Device set to use cuda:0
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


模型推理
保存结果


- 代码说明：
    - 注意输出模型的形状是(n, 2)格式。

### (5) 使用musicgen-stereo-medium模型生成音乐

- 一组能够处理立体声的模型。
    - 立体声音响，也称为立体声，是一种用于再现具有深度和方向感的声音的技术。它使用两个独立的音频通道通过扬声器（或耳机）播放，从而创造出声音来自多个方向的印象。
    - MusicGen是一个文本到音乐的模型，能够根据文本描述或音频提示生成高质量的音乐样本。它是一个单一阶段的自回归Transformer模型，基于32kHz EnCodec分词器进行训练，该分词器有4个以50Hz采样的码本。与现有方法如MusicLM不同，MusicGen不需要自我监督的语义表示，并且可以在一次通过中生成所有4个码本。

In [2]:
import torch
import soundfile as sf
from transformers import pipeline

# 创建
synthesiser = pipeline("text-to-audio", "F:/03Models/musicgen-stereo-medium", device="cuda:0", dtype=torch.float16)
print(synthesiser)
music = synthesiser("Chinese classical guzheng music", forward_params={"max_new_tokens": 256})
print(music)
print(music["audio"][0].shape)
sf.write("musicgen_out.wav", music["audio"][0].T, music["sampling_rate"])

Device set to use cuda:0


{'audio': array([[[-0.04666138, -0.05020142, -0.04058838, ...,  0.06048584,
          0.05670166,  0.05804443],
        [-0.03741455, -0.04107666, -0.02902222, ...,  0.08093262,
          0.07556152,  0.07885742]]], shape=(1, 2, 161920), dtype=float32), 'sampling_rate': 32000}
(2, 161920)


- 代码说明：
    - 音频生成的结果包含"audio"与"sampling_rate"字段，也是保存为音频文件必须的两个参数。
    - 注意音频数据是numpy.ndarray，但是生成的形状是(2, n)，所以需要转置下，形状转换为（n，2）,2表示立体声。

### (6) 使用Ace-Step1.5模型生成音乐

- Ace-Step1.5模型实际是由四个模型协作完成音乐生成的模型。

- Ace-Step1.5模型结构
    - `Qwen3-Embedding-0.6B`：语义理解层(Semantic Understanding)
        - 音乐生成的第一步不是生成声音，而是理解意图。当你输入“赛博朋克风格的重低音，像银翼杀手那样”时，这个模型将这句话转化为一个 768维或1024维的数学向量 (Embeddings)。
    - `acestep-5Hz-lm-1.7B`：全局规划层 (Global Planning)
        - 它接收Qwen传来的向量，然后通过自回归(Autoregressive)的方式，预测出一串语义Token(Semantic Tokens)。
        - 5Hz表示每秒只生成5个Token。
    - `acestep-v15-turbo`：音频合成层 (Acoustic Synthesis)
        - 这是DiT(Diffusion Transformer)模型。
        - turbo后缀意味着：使用了对抗动态位移蒸馏 (Adversarial Dynamic-Shift Distillation)技术训练而来。
        - 它拿着上一步LM给出的“粗糙骨架”，利用扩散算法，通过去噪过程，将其还原成细腻的 Mel-Spectrogram (梅尔频谱图)。
    - `vae (Variational Autoencoder)`：信号解码层 (Signal Decoding)
        - 计算机内部处理的是频谱图（一种类似热力图的图像），VAE的作用就是将频谱图无损地转换成wav音频波形格式。

- Ace-Step1.5模型的创新突破
    - 传统的音乐模型（如 MusicGen）试图用一个Transformer模型生成音乐：既生成音乐的结构（主歌/副歌），也生成音质（乐器纹理）。结果往往是：生成音乐超过一定时间（比如1分钟），生成的音乐结构与乐器声音质量变差（比如乐器声音变得模糊）。

In [1]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from transformers.generation.streamers import BaseStreamer
from diffusers.models import AutoencoderOobleck
import torch
import soundfile as sf

# ===============加载模型=========================
# 1. 加载核心模型：生成音乐编码的模型
print("加载核心模型：acestep-v15-turbo")
model = AutoModel.from_pretrained(
    "F:/03Models/Ace-Step1.5/acestep-v15-turbo",
    trust_remote_code=True,
    attn_implementation="flash_attention_2",   
    dtype=torch.bfloat16  # flash_attention_2只支持bfloat16与float16.
)

model.config._attn_implementation = "flash_attention_2"   # 受用flash_attention_2算法
model = model.to("cuda:0").to(torch.bfloat16)
model.eval()   # 推理模式

# 2. 音频解码模型，把上面音频模型的输出，解码成音频格式，
print("加载音频解码模型：vae")
vae = AutoencoderOobleck.from_pretrained(
    "F:/03Models/Ace-Step1.5/vae"
)
vae = vae.to("cuda:0").to(torch.bfloat16)
vae.eval()

# 3. 理解用户意图的语言大模型
print("加载意图理解模型：Qwen3-Embedding-0.6B【分词器与模型】")
text_tokenizer = AutoTokenizer.from_pretrained(     
    "F:/03Models/Ace-Step1.5/Qwen3-Embedding-0.6B"
)   # 分词器
text_encoder = AutoModel.from_pretrained(
    "F:/03Models/Ace-Step1.5/Qwen3-Embedding-0.6B"
)  # 意图输出。输出的张量被核心模型，用来生成音频。

text_encoder = text_encoder.to("cuda:0").to(torch.bfloat16)
text_encoder.eval()


#///////////////音乐生成参数与意图理解///////////////////////////////
# 1. 音乐生成的参数
# caption="欢快的电子舞曲，重低音"
caption="悠扬中国古筝民乐"
lyrics=""   # 歌词
bpm=128     # 控制音乐节奏快慢（Beats Per Minute，每分钟节拍数）
duration=30   # 生成音乐的时长
batch_size=1  # 可以生成多个音乐，这里我们设置为一个。
# audio_format="flac"   # 音频文件格式，这个本来是使用torchaudio来保存需要使用的，但本代码中还是使用soundfile框架保存的。
# save_dir="./"         # 音频保存路径，我们直接使用当前路径，torchaudio只需要指定路径，音频文件会随机生成。
instruction = "Fill the audio semantic mask based on the given conditions:"  # 这是生成输入需要指定的音乐结构
language = "unknown"  # 生成歌词的语言，我们这里没有生成歌词，所以使用了unknown
sample_rate = 48000   # 音频文件的波特率。

# 2. 音乐意图理解的参数
text_prompt = F"""# Instruction
{instruction}

# Caption
{caption}

# Metas
- bpm: {bpm}
- timesignature: N/A
- keyscale: N/A
- duration: {duration} seconds
<|endoftext|>
"""
print("音频生成的音乐节奏意图理解")
text_inputs_dict = text_tokenizer(
    text_prompt,
    padding="longest",
    truncation=True,
    max_length=256,
    return_tensors="pt",
)
text_hidden_states = text_inputs_dict["input_ids"].to("cuda")
text_attention_mask = text_inputs_dict["attention_mask"].to("cuda")
text_hidden_states = text_encoder(input_ids=text_hidden_states, lyric_attention_mask=None).last_hidden_state

# 3. 歌词的理解
print("音频生成的歌词理解")
lyrics_text = f"# Languages\n{language}\n\n# Lyric\n{lyrics}<|endoftext|>"
lyrics_inputs_dict = text_tokenizer(
    lyrics_text,
    padding="longest",
    truncation=True,
    max_length=2048,
    return_tensors="pt",
)

lyric_hidden_states = lyrics_inputs_dict["input_ids"].to("cuda")
lyric_attention_mask = lyrics_inputs_dict["attention_mask"].to("cuda")
# lyric_hidden_states = text_encoder(input_ids=lyric_hidden_states, lyric_attention_mask=None).last_hidden_state
lyric_hidden_states = text_encoder.embed_tokens(lyric_hidden_states)

# 参考音频
print("加载参考音频")
# refer_audios = [[torch.zeros(2, 30 * sample_rate)] for _ in range(batch_size)]
refer_audio_latents = torch.load("F:/03Models/Ace-Step1.5/acestep-v15-turbo/silence_latent.pt", weights_only=True).transpose(1, 2)
refer_audio_latents = refer_audio_latents.to("cuda").to(torch.bfloat16)
expected_latent_length = duration * sample_rate // 1920   # 1920会影响音频时间长度(而且vea解码内存可能不足够)
# expected_latent_length = duration * sample_rate // 480   # 1920会影响音频时间长度
refer_audio_latents = refer_audio_latents[:, :expected_latent_length, :]
refer_audio_order_mask = torch.tensor([0], device="cuda", dtype=torch.long)    # [0]个数与batch_size一致

chunk_mask = torch.ones(expected_latent_length, dtype=torch.bool, device="cuda")
chunk_mask = chunk_mask.unsqueeze(-1).repeat(1, 1, refer_audio_latents.shape[2])
src_latents = refer_audio_latents.clone()
is_covers=torch.tensor([False], device="cuda", dtype=torch.bool)

#///////////////////生成音频////////////////////////
print("音乐生成......")
outputs = model.generate_audio(
    text_hidden_states=text_hidden_states,
    text_attention_mask=text_attention_mask,
    lyric_hidden_states=lyric_hidden_states,
    lyric_attention_mask=lyric_attention_mask,
    refer_audio_acoustic_hidden_states_packed=refer_audio_latents,
    refer_audio_order_mask=refer_audio_order_mask,
    src_latents=src_latents,
    chunk_masks=chunk_mask,
    is_covers=is_covers
)
#///////////////////音频解码并保存////////////////////////
# 1. 音频解码
print("音频解码")
pred_latents = outputs["target_latents"]
pred_latents_cpu = pred_latents.clone()
pred_latents_for_decode = pred_latents.transpose(1, 2).contiguous()
pred_latents_for_decode = pred_latents_for_decode.to(vae.dtype)

decoder_output = vae.decode(pred_latents_for_decode)
pred_wavs = decoder_output.sample
pred_wavs = decoder_output.sample.detach().to("cpu").float().numpy()
# 2. 保存为wav文件
print("保存音频文件为：AceStep_out.wav")
sf.write("AceStep_out.wav", pred_wavs[0].T, samplerate=sample_rate)

Multiple distributions found for package optimum. Picked distribution: optimum


加载核心模型：acestep-v15-turbo
加载音频解码模型：vae


C:\Program Files\Python313\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


加载意图理解模型：Qwen3-Embedding-0.6B【分词器与模型】
音频生成的音乐节奏意图理解
音频生成的歌词理解
加载参考音频
音乐生成......
音频解码
保存音频文件为：AceStep_out.wav


- 代码说明:
    - 音乐生成最核心的模型是三个：
        - 音乐与歌词意图理解模型：Qwen3-Embedding-0.6B
        - 音乐生成模型：acestep-v15-turbo
        - 音频解码模型：vae
        - 注意：在Ace-Step1.5模型中，还有一个音乐与歌词细节补充模型：acestep-5Hz-lm-1.7B
    - 在vae解码的时候，为了节省内存，可以采用分块解码的方法，我们是直接一次性解码，在本人机器上，大约也就只能生成30-40秒的音频。
    - 本代码仅仅是演示Ace-Step1.5模型的工作步骤与流程，其实还有很多细节可以强化处理。Ace-Step1.5模型还可以完成如下工作：
        - 更加长的音乐生成（acestep-5Hz-lm-1.7B模型：全能规划器模型）。
        - 歌词生成。
        - 翻唱生成。
        - 重绘（repainting）。
        - 人声转背景音乐（BGM）。
        - 支持50多种语言的提示词。

## 4.2. 文本转语音任务的技术分析

- 在技术分析中还是重点关注输入数据的处理，以及输出数据的处理。并能了解模型的网络结构与核心特点。

- 明显语音模型有很多，重点掌握几个经典的模型。

-----